In [38]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

df = pd.read_csv('modified.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,2.128232,0.530628,0.00,1.064711,0.073250,2.484907,3.555348,0.9978,3.51,0.444686,2.341806,5
1,2.174752,0.631272,0.00,1.280934,0.093490,3.258097,4.219508,0.9968,3.20,0.518794,2.379546,5
2,2.174752,0.565314,0.04,1.193922,0.088011,2.772589,4.007333,0.9970,3.26,0.500775,2.379546,5
3,2.501436,0.246860,0.56,1.064711,0.072321,2.890372,4.110874,0.9980,3.16,0.457425,2.379546,6
4,2.128232,0.530628,0.00,1.064711,0.073250,2.484907,3.555348,0.9978,3.51,0.444686,2.341806,5


In [39]:
df.shape

(1599, 12)

In [40]:
df['quality'].value_counts()
def map_quality(val):
    if val in [3,4]:
        return 0
    elif val in [5,6]:
        return 1
    else:
        return 2

df['quality_class'] = df['quality'].apply(map_quality)

In [41]:
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,quality_class
0,2.128232,0.530628,0.00,1.064711,0.073250,2.484907,3.555348,0.99780,3.51,0.444686,2.341806,5,1
1,2.174752,0.631272,0.00,1.280934,0.093490,3.258097,4.219508,0.99680,3.20,0.518794,2.379546,5,1
2,2.174752,0.565314,0.04,1.193922,0.088011,2.772589,4.007333,0.99700,3.26,0.500775,2.379546,5,1
3,2.501436,0.246860,0.56,1.064711,0.072321,2.890372,4.110874,0.99800,3.16,0.457425,2.379546,6,1
4,2.128232,0.530628,0.00,1.064711,0.073250,2.484907,3.555348,0.99780,3.51,0.444686,2.341806,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,1.974081,0.470004,0.08,1.098612,0.086178,3.496508,3.806662,0.99490,3.45,0.457425,2.442347,5,1
1595,1.931521,0.438255,0.10,1.163151,0.060154,3.688879,3.951244,0.99512,3.52,0.565314,2.501436,6,1
1596,1.987874,0.412110,0.13,1.193922,0.073250,3.401197,3.713572,0.99574,3.42,0.559616,2.484907,6,1
1597,1.931521,0.497740,0.12,1.098612,0.072321,3.496508,3.806662,0.99547,3.57,0.536493,2.415914,5,1


In [42]:
df.drop(columns=['quality'], inplace=True)

In [43]:
X = df.drop('quality_class', axis=1)
y = df['quality_class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

num_classes = len(encoder.classes_) 
y_train_cat = to_categorical(y_train_enc, num_classes=num_classes)
y_test_cat = to_categorical(y_test_enc, num_classes=num_classes)

In [48]:

model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train, y_train_cat,
    validation_split=0.2,
    epochs=100,
    batch_size=32
)

test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=0)
print("✅ Test Accuracy:", test_acc)



Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.7879 - loss: 0.6527 - val_accuracy: 0.7891 - val_loss: 0.5954
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8368 - loss: 0.5002 - val_accuracy: 0.8047 - val_loss: 0.5329
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8319 - loss: 0.4550 - val_accuracy: 0.8008 - val_loss: 0.5203
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8328 - loss: 0.4383 - val_accuracy: 0.8164 - val_loss: 0.5100
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8436 - loss: 0.4291 - val_accuracy: 0.8047 - val_loss: 0.5168
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8436 - loss: 0.4316 - val_accuracy: 0.8047 - val_loss: 0.5059
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8328 - loss: 0.4233 - val_accuracy: 0.8125 - val_loss: 0.4954
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8563 - loss: 0.4048 - val_accuracy: 0.8164 - 